# SentencePiece

토큰화의 끝판왕으로 보이는 이 WPM은 아쉽게도 공개되어 있지는 않습니다. 대신에 구글의 *SentencePiece* 라이브러리를 통해 고성능의 BPE를 사용할 수 있습니다! *SentencePiece*에는 전처리 과정도 포함되어 있어서, 데이터를 따로 정제할 필요가 없어 간편하기까지 합니다. 따라서 아래 깃허브 페이지에 방문해 사용법을 간단히 눈으로 봐두도록 하죠!  아마 다음 프로젝트 노드에서는 *SentencePiece* 라이브러리를 적극적으로 활용하는 실습을 진행하게 될 것입니다. 

- [google/sentencepiece](https://github.com/google/sentencepiece)

이제 우리는 어떤 언어에도 OOV 발생 우려 없이 안정적으로 활용할 수 있는 멋진 토크나이징 기술을 확보했습니다. 이제는 컴퓨터가 단어사전을 안심하고 활용할 수 있겠군요!

하지만 아직 우리는 제대로 된 단어의 분산 표현을 얻는 법을 제대로 다루지는 않았습니다. 이쯤에서 이런 고민이 생기게 됩니다. 한국어라면 `자동차`를 `_자동 / 차` 로 분리되는데... 속성이 아무리 추상적이래도 보기에 `차`가 마시는 차인지, 달리는 차인지 도통 알 수가 없죠? 게다가 설령 토큰화가 완벽하다고 해도, `남자`가 `[-1, 0]`인지 `[1, 0]`인지는 컴퓨터 입장에서는 알 도리가 없습니다.

Embedding 레이어는 선언 즉시 랜덤한 실수로 Weight 값을 채우고, 학습을 진행하며 적당히 튜닝해가는 방식으로 속성을 맞춰가지만 이는 뭔가 찜찜합니다. 토큰들이 멋지게 의미를 갖게 하는 방법은 없을까요?

# IMDB로 센텐스피스 적용하기

In [ ]:
!pip install sentencepiece

In [ ]:
import sentencepiece as spm
import pandas as pd
import urllib.request
import csv

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/LawrenceDuan/IMDb-Review-Analysis/master/IMDb_Reviews.csv", filename="IMDb_Reviews.csv")

In [ ]:
# pd.read_csv로 csv를 읽어주세요.
train_df = None

In [ ]:
# train_df의 상위 5개를 출력해주세요.
train_df.None

In [ ]:
print("리뷰 갯수 :", len(train_df))

In [ ]:
with open('imdb_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_df['review']))

```
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰
```
```
spm.SentencePieceTrainer.Train('--input=imdb_review.txt --model_prefix=imdb --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')
```

In [ ]:
corpus = 'imdb_review.txt' # 입력 corpus
prefix = 'imdb' # 저장할 단어장 이름
vocab_size = 5000
spm.SentencePieceTrainer.Train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size}" +
    "--model_type=bpe" +
    "--max_sentence_length=999999" #문장 최대 길이
)

In [ ]:
vocab_list = pd.read_csv('imdb.vocab', sep='\t', header = None, quoting=csv.QUOTE_NONE)
vocab_list.sample(10)

In [ ]:
# vocab_list의 길이를 출력해주세요


In [ ]:
sp = spm.SentencePieceProcessor()
vocab_file = 'imdb.model'
sp.load(vocab_file)

In [ ]:
lines = [
         "I didn't at all think of it this way.",
         "I have waited a long time for someone to film"
]

for line in lines:
    print(line)
    print(sp.encode_as_pieces(line)) # 서브워드 시퀀스 변환
    print(sp.encode_as_ids(line)) # 정수 시퀀스 변환
    print()

In [ ]:
sp.GetPieceSize() # 단어집합의 크기

In [ ]:
sp.IdToPiece(120) # 정수로부터 매핑되는 서브워드 변환

In [ ]:
sp.PieceToId('▁will') # 대문자 I, 서브워드로부터 매핑되는 정수로 변환

In [ ]:
sp.DecodeIds([None])
# 정수 시퀀스를 sp.DecodeIds의 정수 시퀀스에 입력

In [ ]:
sp.DecodePieces([None])
# 서브워드 시퀀스로부터 문장으로 변환

In [ ]:
print(sp.encode('I have waited a long time for someone to film', out_type=str)) # subword 시퀀스
print(sp.encode('I have waited a long time for someone to film', out_type=int)) # 정수 시퀀스

# 네이버 영화리뷰로 센텐스피스 적용하기

In [ ]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

In [ ]:
# naver_df 변수에 pd.read_table로 csv를 저장하세요
naver_df = None

In [ ]:
# naver_df의 상위 5개를 출력하세요~
None

In [ ]:
# naver_df의 갯수를 출력하세요.
None

In [ ]:
# NULL값이 존재하는지 확인하세요.
None

In [ ]:
# Null값이 존재하는 행 제거
None

In [ ]:
# Null값이 제거가 되었는지, 다시한번 Null값이 존재하는지 확인하세요~
None

In [ ]:
# naver_df의 리뷰갯수를 확인하세요!
None

In [ ]:
# Quiz
# naver data 'document'컬럼을 txt에 새로 만드세요.


In [ ]:
# Quiz
# sentencePieceTrainer로 학습


In [ ]:
# Quiz
# vocab을 불러오기


In [ ]:
# Quiz
# vocab모델이 있는지 확인

In [ ]:
lines = [
         "뭐 이딴 것도 영화냐.",
         "진짜 최고의 영화입니다ㅋㅋ",
]

for line in lines:
    print(line)
    print(None) # 서브워드로 변환
    print(None) # 정수 시퀀스로 변환
    print()

In [ ]:
# 단어집합의 크기를 확인하세요.
sp.None

In [ ]:
# 정수로부터 매핑되는 서브워드 변환
sp.None

In [ ]:
# 대문자 I, 서브워드로부터 매핑되는 정수로 변환
sp.None('▁공')

In [ ]:
# 정수 시퀀스를 sp.DecodeIds의 정수 시퀀스에 입력
sp.None([None])

In [ ]:
# 서브워드 시퀀스로부터 문장으로 변환
sp.DecodePieces(['▁뭐', '▁이딴', '▁것도', '▁영화냐', '.'])

In [ ]:
# out_type의 옵션을 변경하면서 결과를 확인해보세요. out_type에서 제공하는 옵션은 str, int입니다.

# subword 시퀀스
print(sp.encode('진짜 최고의 영화입니다ㅋㅋ', out_type=None))
# 정수 시퀀스
print(sp.encode('진짜 최고의 영화입니다ㅋㅋ', out_type=None))